#<h1>Travelling Salesman Problem</h1>
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1vEYijQlc4QRn-ZnW9LYVotI9cgvLWZAy?usp=sharing">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/AlexandroLuis/Hyper-heuristic_to_GeneticAlgorithms/blob/main/Travelling%20Salesman%20Problem/TSP_Without_Hyper-Heuristic.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>
<br><br>
<br>

##Import Libraries

In [ ]:
!pip install jmetalpy
!pip install thompson-sampling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import math
import random
import re
import json
import os
import time
import shutil
import csv
import threading
import copy

from abc import ABC, abstractmethod
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from pandas import Series

from typing import Generic, List, TypeVar

import jmetal
import logging.config

#operator
from jmetal.operator import *
from jmetal.operator.mutation import PermutationSwapMutation
from jmetal.operator.crossover import PMXCrossover, Crossover

# util
from jmetal.util.ranking import StrengthRanking
from jmetal.util.density_estimator import KNearestNeighborDensityEstimator
from jmetal.util.observer import ProgressBarObserver
from jmetal.util.solution import get_non_dominated_solutions
from jmetal.util import termination_criterion
from jmetal.util.termination_criterion import TerminationCriterion, StoppingByEvaluations
from jmetal.util.comparator import Comparator, DominanceComparator, MultiComparator
from jmetal.util.evaluator import Evaluator
from jmetal.util.generator import Generator

#core
from jmetal.core.quality_indicator import *
from jmetal.core.problem import PermutationProblem, Problem
from jmetal.core.solution import PermutationSolution, Solution, FloatSolution
from jmetal.core.operator import Mutation, Crossover, Selection

#lab
from jmetal.lab.experiment import Experiment, Job, generate_summary_from_experiment

#config
from jmetal.config import store

#thompson-sampling
from thompson_sampling.bernoulli import BernoulliExperiment
from thompson_sampling.priors import BetaPrior


##Import Problem

###Import TSP Instances

In [ ]:
# for windows put Desktop+path
windows_path = "Desktop"
instances_path = "/content/instances/" 
output_path = windows_path+"/content/output/"
instances_zip_path = "/content/instances.zip"

In [ ]:
try:
  os.mkdir(instances_path)
except:
  pass

# Download instances from a google drive folder
try:
  !gdown 1SN-p8Wj6z0hCshiueZety4dEhkjlhDTr
except Exception as e:
  print(e)

# Unpack it
try:
  os.replace(instances_zip_path, instances_path+"instances.zip")
  shutil.unpack_archive(instances_path+"instances.zip", instances_path)
  os.remove(instances_path+"instances.zip")
except Exception as e:
  print(f"error: {e}")

Downloading...
From: https://drive.google.com/uc?id=1SN-p8Wj6z0hCshiueZety4dEhkjlhDTr
To: /content/instances.zip
100% 74.7k/74.7k [00:00<00:00, 75.7MB/s]


###Reading TSP Files

In [ ]:
def read_tsplib_file(filename):
    if filename is None:
        raise FileNotFoundError('Filename can not be None')
    with open(filename) as file:
        lines = file.readlines()
        data = [line.lstrip() for line in lines if line != ""]
        dimension = re.compile(r'[^\d]+')
        for item in data:
            if item.startswith('DIMENSION'):
                dimension = int(dimension.sub('', item))
                break
        c = [-1.0] * (2 * dimension)
        cities_coord = []
        for item in data:
            if item[0].isdigit():
                j, coordX, coordY = [float(x.strip()) for x in item.split(' ')]
                c[2 * (int(j) - 1)] = coordX
                c[2 * (int(j) - 1) + 1] = coordY
                cities_coord.append([coordX,coordY])
        cities = pd.DataFrame(cities_coord)
        matrix = [[-1] * dimension for _ in range(dimension)]
        for k in range(dimension):
            matrix[k][k] = 0
            for j in range(k + 1, dimension):
                dist = math.sqrt((c[k * 2] - c[j * 2]) ** 2 + (c[k * 2 + 1] - c[j * 2 + 1]) ** 2)
                dist = round(dist)
                matrix[k][j] = dist
                matrix[j][k] = dist
        return matrix, dimension, cities

###TSP Solver Class

In [ ]:
class myTSP(PermutationProblem):
    def __init__(self, distance_matrix, number_of_cities, fitness_log):
        super(myTSP, self).__init__()
        self.distance_matrix = distance_matrix
        self.number_of_variables = number_of_cities
        self.obj_directions = [self.MINIMIZE]
        self.number_of_objectives = 1
        self.number_of_constraints = 0
        self.fitness_log = fitness_log
        
    def evaluate(self, solution: PermutationSolution) -> PermutationSolution:
        fitness = 0
        for i in range(self.number_of_variables - 1):
            x = solution.variables[i]
            y = solution.variables[i + 1]
            fitness += self.distance_matrix[x][y]
        first_city, last_city = solution.variables[0], solution.variables[-1]
        fitness += self.distance_matrix[first_city][last_city]
        solution.objectives[0] = fitness
        self.fitness_log.append(fitness)
        return solution
    
    def create_solution(self) -> PermutationSolution:
        new_solution = PermutationSolution(number_of_variables=self.number_of_variables,
                                           number_of_objectives=self.number_of_objectives)
        new_solution.variables = random.sample(range(self.number_of_variables), k=self.number_of_variables)
        return new_solution

    @property
    def number_of_cities(self):
        return self.number_of_variables

    def get_name(self):
        return 'Symmetric TSP'

##Load Problem Informations

In [ ]:
class problem():
  def __init__(self, j):
        self.problem_name = j
        self.optimal_fitness = 0000 # it return the optimum of each different problem
        self.location = instances_path+j
        self.dist_matrix, self.nb_cities, self.cities_coord = read_tsplib_file(self.location)

  def myProblem(self, fitness_log): 
    return myTSP(self.dist_matrix, self.nb_cities, fitness_log)

  def getProblemName(self):
    return self.problem_name

  def getOptimalFitness(self):
    return self.optimal_fitness
  
  def getCityCoord(self):
    return self.cities_coord
  
  def getDimensions(self):
    return self.nb_cities

##Operators Core

### Informations Catch

In [ ]:
class infoCatch():
  def __init__(self):
    self.eachSonOP = []
    self.evenOPCounter = [0, 0, 0, 0]

  def updateInfo(self, values, operator):
    try:
      for i in range(2):
        self.eachSonOP.append(
            {'offspring and operator': [
                  {
                    'offspring '+str(i+1): values[i], 
                    'operator ': operator
                  }
                ]
             }
        )

      x = ["Scramble&PMX", "Scramble&CX", "Swap&PMX", "Swap&CX"]
      for i, j in zip(x, range(len(x))):
        if str(i) == str(operator):
          self.evenOPCounter[j] = self.evenOPCounter[j]+1
          break

    except Exception as e:
      print(f"Error while collecting infos: {e}")

  def getInfo(self):
    print(f"\noperadores por pai: {self.eachSonOP}\n")
    print(f"\ncontador:{self.evenOPCounter}\n")

###Logging

In [ ]:
def get_logger(module):
    return logging.getLogger(module)

logger = get_logger(__name__)

S = TypeVar("S")
R = TypeVar("R")

def configure_logging():
    DEFAULT_LOGGING_CONFIG = {
        "version": 1,
        "disable_existing_loggers": False,
        "formatters": {
            "basic": {
                "format": "[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s",
            }
        },
        "handlers": {
            "console": {
                "formatter": "basic",
                "class": "logging.StreamHandler",
                "stream": "ext://sys.stderr",
            }
        },
        "loggers": {
            "jmetal": {"handlers": ["console"], "level": "DEBUG"},
        },
    }

    logging.config.dictConfig(DEFAULT_LOGGING_CONFIG)


### Thompson Sampling

In [ ]:
class ThompsonSampling():
  def __init__(self):
    self.means = Series([0.5, 0.5, 0.5, 0.5])
    self.variances = Series([0.1, 0.1, 0.1, 0.1])
    self.effective_sizes = Series([10, 10, 10, 10])
    self.labels = Series(["Scramble&PMX", "Scramble&CX", "Swap&PMX", "Swap&CX"])

    pr = BetaPrior()
    pr.add_multiple(self.means, self.variances, self.effective_sizes, self.labels)
    
    self.experiment = BernoulliExperiment(priors = pr)

  def TS_action(self):
    arms = self.experiment.choose_arm()
    return arms
  
  def TS_update(self, Result, mutation_operator, crossover_operator, operator):
    try:
      rewards = []
      if Result:
        for i in ["Scramble&PMX", "Scramble&CX", "Swap&PMX", "Swap&CX"]:
          if str(i) == str(operator):
            rewards.append({"label":i, "reward":1})
          else:
            rewards.append({"label":i, "reward":0})
      else:
        for i in ["Scramble&PMX", "Scramble&CX", "Swap&PMX", "Swap&CX"]:
          if str(i) == str(operator):
            rewards.append({"label":i, "reward":-1})
          else:
            rewards.append({"label":i, "reward":0})

      self.experiment.add_rewards(rewards)
    except Exception as e:
      print(f"Thompson Sampling Class Error: {e}\n") 

  def get_result(self):
    print(self.experiment.get_ppd(size=2000))

###Heuristics

In [ ]:
'''
  Start the LKH algorithm with the current tour.
'''
#http://akira.ruc.dk/~keld/research/LKH/LKH-1.3/DOC/LKH_REPORT.pdf

class LKH_Heuristic():
  def __init__(self, result):
    self.tour = result.variables
    pass

  def execution():
    pass
    


###Selection

In [ ]:
"""
  Performs roulette wheel selection.
""" 

S = TypeVar('S')

class RouletteWheelSelection(Selection[List[S], S]):
  def __init__(self):
    super(RouletteWheelSelection).__init__()

  def execute(self, front: List[S]) -> S:
    if front is None:
        raise Exception('The front is null')
    elif len(front) == 0:
        raise Exception('The front is empty')

    maximum = sum([solution.objectives[0] for solution in front])
    rand = random.uniform(0.0, maximum)
    value = 0.0

    for solution in front:
        value += solution.objectives[0]

        if value > rand:
            return solution

    return None

  def get_name(self) -> str:
    return 'Roulette wheel selection'

###Crossover

In [ ]:
class CXCrossover(Crossover[PermutationSolution, PermutationSolution]):
  def __init__(self, probability: float):
    super(CXCrossover, self).__init__(probability=probability)
  def execute(self, parents: List[PermutationSolution]) -> List[PermutationSolution]:
    if len(parents) != 2:
      raise Exception('The number of parents is not two: {}'.format(len(parents)))

    offspring = [copy.deepcopy(parents[1]), copy.deepcopy(parents[0])]

    rand = random.random()

    if rand <= self.probability:
      idx = random.randint(0, len(parents[0].variables) - 1)
      curr_idx = idx
      cycle = []

      while True:
        cycle.append(curr_idx)
        curr_idx = parents[0].variables.index(parents[1].variables[curr_idx])

        if curr_idx == idx:
          break

      for j in range(len(parents[0].variables)):
        if j in cycle:
          offspring[0].variables[j] = parents[0].variables[j]
          offspring[1].variables[j] = parents[0].variables[j]
      
    return offspring

  def get_number_of_parents(self) -> int:
    return 2

  def get_number_of_children(self) -> int:
    return 2

  def get_name(self):
    return 'Cycle crossover'


In [ ]:
class PMXCrossover(Crossover[PermutationSolution, PermutationSolution]):
    def __init__(self, probability: float):
        super(PMXCrossover, self).__init__(probability=probability)

    def execute(self, parents: List[PermutationSolution]) -> List[PermutationSolution]:
        if len(parents) != 2:
            raise Exception('The number of parents is not two: {}'.format(len(parents)))

        offspring = [copy.deepcopy(parents[0]), copy.deepcopy(parents[1])]
        permutation_length = offspring[0].number_of_variables

        rand = random.random()
        if rand <= self.probability:
            cross_points = sorted([random.randint(0, permutation_length) for _ in range(2)])

            def _repeated(element, collection):
                c = 0
                for e in collection:
                    if e == element:
                        c += 1
                return c > 1

            def _swap(data_a, data_b, cross_points):
                c1, c2 = cross_points
                new_a = data_a[:c1] + data_b[c1:c2] + data_a[c2:]
                new_b = data_b[:c1] + data_a[c1:c2] + data_b[c2:]
                return new_a, new_b

            def _map(swapped, cross_points):
                n = len(swapped[0])
                c1, c2 = cross_points
                s1, s2 = swapped
                map_ = s1[c1:c2], s2[c1:c2]
                for i_chromosome in range(n):
                    if not c1 < i_chromosome < c2:
                        for i_son in range(2):
                            while _repeated(swapped[i_son][i_chromosome], swapped[i_son]):
                                map_index = map_[i_son].index(swapped[i_son][i_chromosome])
                                swapped[i_son][i_chromosome] = map_[1 - i_son][map_index]
                return s1, s2

            swapped = _swap(parents[0].variables, parents[1].variables, cross_points)
            mapped = _map(swapped, cross_points)

            offspring[0].variables, offspring[1].variables = mapped

        return offspring

    def get_number_of_parents(self) -> int:
        return 2

    def get_number_of_children(self) -> int:
        return 2

    def get_name(self):
        return 'Partially Matched crossover'

###Mutation

In [ ]:
class ScrambleMutation(Mutation[PermutationSolution]):
  def execute(self, solution: PermutationSolution) -> PermutationSolution:
    rand = random.random()

    if rand <= self.probability:
      point1 = random.randint(0, len(solution.variables))
      point2 = random.randint(0, len(solution.variables) - 1)

      if point2 >= point1:
        point2 += 1
      else:
        point1, point2 = point2, point1

      if point2 - point1 >= 20:
        point2 = point1 + 20

      values = solution.variables[point1:point2]
      solution.variables[point1:point2] = random.sample(values, len(values))

    return solution


  def get_name(self):
    return 'Scramble Mutation'

In [ ]:
'''
class PermutationSwapMutation(Mutation[PermutationSolution]):

    def execute(self, solution: PermutationSolution) -> PermutationSolution:
        Check.that(type(solution) is PermutationSolution, "Solution type invalid")

        rand = random.random()

        if rand <= self.probability:
            pos_one, pos_two = random.sample(range(solution.number_of_variables - 1), 2)
            solution.variables[pos_one], solution.variables[pos_two] = \
                solution.variables[pos_two], solution.variables[pos_one]

        return solution

    def get_name(self):
        return 'Permutation Swap mutation'
'''

'\nclass PermutationSwapMutation(Mutation[PermutationSolution]):\n\n    def execute(self, solution: PermutationSolution) -> PermutationSolution:\n        Check.that(type(solution) is PermutationSolution, "Solution type invalid")\n\n        rand = random.random()\n\n        if rand <= self.probability:\n            pos_one, pos_two = random.sample(range(solution.number_of_variables - 1), 2)\n            solution.variables[pos_one], solution.variables[pos_two] =                 solution.variables[pos_two], solution.variables[pos_one]\n\n        return solution\n\n    def get_name(self):\n        return \'Permutation Swap mutation\'\n'

###Algorithm

In [ ]:
class Algorithm(Generic[S, R], threading.Thread, ABC):
    def __init__(self):
        threading.Thread.__init__(self)

        self.solutions: List[S] = []
        self.evaluations = 0
        self.start_computing_time = 0
        self.total_computing_time = 0

        self.observable = store.default_observable

    @abstractmethod
    def create_initial_solutions(self) -> List[S]:
        """Creates the initial list of solutions of a metaheuristic."""
        pass

    @abstractmethod
    def evaluate(self, solution_list: List[S]) -> List[S]:
        """Evaluates a solution list."""
        pass

    @abstractmethod
    def init_progress(self) -> None:
        """Initialize the algorithm."""
        pass

    @abstractmethod
    def stopping_condition_is_met(self) -> bool:
        """The stopping condition is met or not."""
        pass

    @abstractmethod
    def step(self) -> None:
        """Performs one iteration/step of the algorithm's loop."""
        pass

    @abstractmethod
    def update_progress(self) -> None:
        """Update the progress after each iteration."""
        pass

    @abstractmethod
    def get_observable_data(self) -> dict:
        """Get observable data, with the information that will be send to all observers each time."""
        pass

    def run(self):
        """Execute the algorithm."""
        self.start_computing_time = time.time()

        logger.debug("Creating initial set of solutions...")
        self.solutions = self.create_initial_solutions()

        logger.debug("Evaluating solutions...")
        self.solutions = self.evaluate(self.solutions)

        logger.debug("Initializing progress...")
        self.init_progress()

        logger.debug("Running main loop until termination criteria is met")

        while not self.stopping_condition_is_met():
            self.step()
            self.update_progress()

        logger.debug("Finished!")
        self.total_computing_time = time.time() - self.start_computing_time

    @abstractmethod
    def get_result(self) -> R:
        pass

    @abstractmethod
    def get_name(self) -> str:
        pass

In [ ]:
class DynamicAlgorithm(Algorithm[S, R], ABC):
    @abstractmethod
    def restart(self) -> None:
        pass

In [ ]:
class EvolutionaryAlgorithm(Algorithm[S, R], ABC):
    def __init__(self, problem: Problem[S], population_size: int, offspring_population_size: int):
        super(EvolutionaryAlgorithm, self).__init__()
        self.problem = problem
        self.population_size = population_size
        self.offspring_population_size = offspring_population_size

    @abstractmethod
    def selection(self, population: List[S]) -> List[S]:
        """Select the best-fit individuals for reproduction (parents)."""
        pass

    @abstractmethod
    def reproduction(self, population: List[S]) -> List[S]:
        """Breed new individuals through crossover and mutation operations to give birth to offspring."""
        pass

    @abstractmethod
    def replacement(self, population: List[S], offspring_population: List[S]) -> List[S]:
        """Replace least-fit population with new individuals."""
        pass

    def get_observable_data(self) -> dict:
        return {
            "PROBLEM": self.problem,
            "EVALUATIONS": self.evaluations,
            "SOLUTIONS": self.get_result(),
            "COMPUTING_TIME": time.time() - self.start_computing_time,
        }

    def init_progress(self) -> None:
        self.evaluations = self.population_size

        observable_data = self.get_observable_data()
        self.observable.notify_all(**observable_data)

    def step(self):
        mating_population = self.selection(self.solutions)
        offspring_population = self.reproduction(mating_population)
        offspring_population = self.evaluate(offspring_population)

        self.solutions = self.replacement(self.solutions, offspring_population)

    def update_progress(self) -> None:
        self.evaluations += self.offspring_population_size

        observable_data = self.get_observable_data()
        self.observable.notify_all(**observable_data)

    @property
    def label(self) -> str:
        return f"{self.get_name()}.{self.problem.get_name()}"

In [ ]:
class GeneticAlgorithm(EvolutionaryAlgorithm[S, R]):
    def __init__(
                 self,
                 problem: Problem,
                 population_size: int,
                 offspring_population_size: int,
                 mutation: Mutation,
                 crossover: Crossover,
                 selection: Selection,
                 termination_criterion: TerminationCriterion = store.default_termination_criteria,
                 population_generator: Generator = store.default_generator,
                 population_evaluator: Evaluator = store.default_evaluator
        ):
        super(GeneticAlgorithm, self).__init__(problem=problem, population_size=population_size, offspring_population_size=offspring_population_size)
        # Mutation
        self.mutation = mutation
        self.mutation_operator = NullMutation()
        # Crossover
        self.crossover = crossover
        self.crossover_operator = NullCrossover()
        #Selection
        self.selection_operator = selection

        self.population_generator = population_generator
        self.population_evaluator = population_evaluator

        self.termination_criterion = termination_criterion
        self.observable.register(termination_criterion)

        # Thompson Sampling init
        self.thompsonsampling = ThompsonSampling()
        self.infoCatch = infoCatch()

        self.mating_pool_size = \
            self.offspring_population_size * \
            self.crossover_operator.get_number_of_parents() // self.crossover_operator.get_number_of_children()

        if self.mating_pool_size < self.crossover_operator.get_number_of_children():
            self.mating_pool_size = self.crossover_operator.get_number_of_children()


    def create_initial_solutions(self) -> List[S]:
        return [self.population_generator.new(self.problem)
                for _ in range(self.population_size)]

    def evaluate(self, population: List[S]):
        return self.population_evaluator.evaluate(population, self.problem)

    def thompson_sampling_updateOperator(self):
      self.operator = self.thompsonsampling.TS_action()
      if self.operator[0:4] == "Swap":
        self.mutation_operator = self.mutation[1]
        if self.operator[:-2] == "CX":
          self.crossover_operator = self.crossover[1]
        else:
          self.crossover_operator = self.crossover[0]
      else:
        self.mutation_operator = self.mutation[0]
        if self.operator[-2:] == "CX":
          self.crossover_operator = self.crossover[1]
        else:
          self.crossover_operator = self.crossover[0]

    def thompson_sampling_updateRewards(self, parents, generated):
      try:
        generated = self.evaluate(generated)
        if generated[0].objectives[0] or generated[1].objectives[0] > parents[0].objectives[0] or parents[1].objectives[0]:
          Result = True
        else:
          Result = False

        self.thompsonsampling.TS_update(
            Result, 
            self.mutation_operator, 
            self.crossover_operator, 
            self.operator
        )
      except Exception as e:
        print(f"Error doing TS update: {e}")
      

    def stopping_condition_is_met(self) -> bool:
        return self.termination_criterion.is_met

    def selection(self, population: List[S]):
        mating_population = []

        for i in range(self.mating_pool_size):
            solution = self.selection_operator.execute(population)
            mating_population.append(solution)

        return mating_population

    def reproduction(self, mating_population: List[S]) -> List[S]:
        number_of_parents_to_combine = self.crossover_operator.get_number_of_parents()

        if len(mating_population) % number_of_parents_to_combine != 0:
            raise Exception('Wrong number of parents')

        offspring_population = []
        for i in range(0, self.offspring_population_size, number_of_parents_to_combine):
            parents = []
            generated = []
            myDict = []

            for j in range(number_of_parents_to_combine):
                parents.append(mating_population[i + j])

            self.thompson_sampling_updateOperator()
            offspring = self.crossover_operator.execute(parents)
            for solution in offspring:
                self.thompson_sampling_updateOperator()
                self.mutation_operator.execute(solution)

                generated.append(solution)
                offspring_population.append(solution)
                myDict.append(solution.variables)

                if len(offspring_population) >= self.offspring_population_size:
                    break
            self.infoCatch.updateInfo(myDict, self.operator)
            self.thompson_sampling_updateRewards(parents, generated)
        return offspring_population

    def replacement(self, population: List[S], offspring_population: List[S]) -> List[S]:
        population.extend(offspring_population)

        population.sort(key=lambda s: s.objectives[0])

        return population[:self.population_size]

    def get_result(self) -> R:
        return self.solutions[0]

    def get_name(self) -> str:
        return 'Genetic algorithm'
    
    def get_TS_result(self):
      return self.thompsonsampling.get_result(), self.infoCatch.getInfo()

##Output

###Save Output to Textual Files

In [ ]:
class bcolors:
  OKGREEN = '\033[32m'
  WARNING = '\033[31m'
  LightBl = "\033[94m"
  ENDC = '\033[0m'

In [ ]:
class outputFile():
  def __init__(self, my_algo, pop_evolved, log, params, i, maxevals, j):
    self.algorithm_name = my_algo.get_name()
    self.solution_x = pop_evolved.variables
    self.fitness = pop_evolved.objectives[0]
    self.n_evals = my_algo.evaluations
    self.duration = my_algo.total_computing_time
    self.lensolution = self.solution_x
    self.params = params
    self.i = i
    self.maxevals = maxevals
    self.d = {}
    self.j = j
    self.path = ""

  def create_output_file(self):
    x = problem(self.j)

    self.d['Function'] = x.getProblemName()
    self.d['Problem dimension'] = self.lensolution
    self.d['Global Optimum'] = x.getOptimalFitness()
    self.d['Algorithm'] = self.algorithm_name
    self.d['Parameters'] = self.params
    self.d['Fitness'] = self.fitness
    self.d['Solution'] = self.solution_x
    self.d['Nb of functions evaluations'] = self.n_evals
    self.d['Computational time in seconds'] = self.duration
    self.d['Stopping criterion in evaluations'] = self.maxevals

    self.calltosave()

  def calltosave(self):
    try:
      self.x = problem(self.j)
      filename = output_path
      filename2 = self.x.getProblemName()
      self.path = os.path.join(filename, filename2)
      Path(self.path).mkdir(parents=True, exist_ok=True)     
    except:
      pass

    try:
      with open("%s/run_%s.txt" %(self.path, self.i), 'w', encoding='utf-8') as f:
        f.write(""+self.i+"° run output:\n")
        json.dump(self.d, f, indent=4, separators=(',',': '))
      f.close()
    except Exception as e:
      print(e)

  def saveTS_output(self, operator, counter, resultLogs):
    try:
      with open("%s/TS-E_RUN_%s.txt" %(self.path, self.i), 'w', encoding='utf-8') as f:
        f.write(""+self.i+"° run output:\n")
        f.write("Result Logs:"+operator+"\n")
        f.write("operator logs:"+operator+"\n")
      f.close()
    except Exception as e:
      print(e)
      pass
    except Exception as e:
      print(e)
  
  def outputFitness(self, fitness, i):
    try:
      with open("%s/fitnessOutput.FUN" %(self.path), 'w', encoding='utf-8') as f:
        f.write('\n'.join(str(line) for line in fitness))
      f.close()
    except Exception as e:
      print(e)

  def getFitness(self):
    return self.fitness
  
  def outputAllFitness(self, output_result, i):
    try:
      try:
        if i == 0:
          # Clean the file 
          with open("%s/routeOutput.VAR" %(self.path), 'w', encoding='utf-8') as f:
            f.close()
      except:
        pass
      # Rewrite non deleting written text
      with open("%s/routeOutput.VAR" %(self.path), 'a', encoding='utf-8') as f:
        f.write(str(output_result))
        f.write('\n')
      f.close()  
    except Exception as e:
      print(e)

  def output_fitness_CSV(self, fitness):
    matrix = [[0] for i in range(len(fitness))]
    for i in range(len(fitness)):
      matrix[i][0] = fitness[i]

    try:
      with open("%s/fitnessOutput.csv" %(self.path), 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        #writer.writerow(header)
        for i in matrix:
          writer.writerow(i)
    except Exception as e:
      print(f"Erro: {e}")

###Plot The Output Result

In [ ]:
#Last Element Output
class lastOutput():
  def __init__(self, fitness_log, result, run):
        self.fitness_log = fitness_log
        self.result = result
        self.runs = 1

  # print graph
  def plotMap(self):
    try:
      plt.figure(figsize=(15,4))
      plt.plot(self.fitness_log[::1000], color="red")
      plt.xlabel("evaluations (x1000)")
      plt.ylabel("fitness")
      plt.show()
    except Exception as e:
      print(f"Error {bcolors.WARNING}{e}{bcolors.ENDC}, Could not plot the fitbness by evaluations")

  # print map
  def printMap(self):
    try:
      cities_coord = problem().getCityCoord()
      xlist = [cities_coord.iloc[i,0] for i in self.result.variables]
      ylist = [cities_coord.iloc[i,1] for i in self.result.variables]

      xlist.append(xlist[0])
      ylist.append(ylist[0])

      plt.figure(figsize=(20,10))
      for idx,city in enumerate(cities_coord.values):
        plt.scatter(city[0],city[1])
        plt.text(city[0]-20, city[1]+40, str(idx), fontsize=10)

      plt.plot(xlist, ylist, color="purple",linestyle='-')
      plt.plot(xlist, ylist, linestyle='-')
      plt.axis('scaled')
      plt.show()
    except Exception as e:
      print(f"Error {bcolors.WARNING}{e}{bcolors.ENDC}, Could not plot the map")

  #run all functions
  def getResults(self):
    if __name__ == '__main__':
      self.plotMap()
      self.printMap()

##Execution

###Genectic Algorithm 

In [ ]:
class TSP():
  def __init__(self, maxgenerations, popsize, offspring, mut_prob, cross_prob, runs):
    # genetic algorithm params
    self.maxgenerations = maxgenerations
    self.popsize = popsize
    self.offspring = offspring
    self.mut_prob = 0
    self.cross_prob = cross_prob
    self.run = runs
    # log params
    self.fitness_log = []
    self.fitness = []
    # function to auto start executions
    self.start()

  def start(self):
    for self.j in os.listdir(instances_path):
      try:
        if self.j[-4:] == ".tsp":
          print(f"{bcolors.WARNING}\nNow Running: {self.j[:-4]}{bcolors.ENDC}\n")
          for self.i in range(self.run):
            print(f"{bcolors.OKGREEN}\nrun {self.i+1}{bcolors.ENDC}")
            self.execute()
          print("\n")
          print(f"{bcolors.LightBl}-{bcolors.ENDC}"*40)
          self.x.outputFitness(self.fitness, self.i)
          self.x.output_fitness_CSV(self.fitness)
          self.fitness.clear()
          #self.output() ##~~ Don't plot the map ~~##  
      except Exception as e: 
        print(f"Error {bcolors.WARNING}{e}{bcolors.ENDC}, Moving to next instance")       
    
  def execute(self):
    problemInfo = problem(self.j)
    mut_prob = 1/problemInfo.getDimensions()

    # AG algorithm
    algorithm = GeneticAlgorithm(
      problem = problemInfo.myProblem(self.fitness_log),
      population_size = self.popsize,
      offspring_population_size = self.offspring,
      mutation = [
          ScrambleMutation(mut_prob), 
          PermutationSwapMutation(mut_prob)
      ], 
      crossover = [
          PMXCrossover(self.cross_prob), 
          CXCrossover(self.cross_prob)
      ],
      selection = RouletteWheelSelection(),
      termination_criterion = termination_criterion.StoppingByEvaluations(max_evaluations=self.maxgenerations),
    ) 

    params = {
              'population': self.popsize, 
  	          'offspring': self.offspring, 
	            'mutation probability': mut_prob, 
		          'crossover probability': self.cross_prob,
  		  }

    # Call GeneticAlgorithm and execute it #
    algorithm.observable.register(ProgressBarObserver(max=self.maxgenerations))
    algorithm.run()
    
    self.result = algorithm.get_result()
    
    self.x = outputFile(algorithm, self.result, self.fitness_log, params, str(self.i+1), self.maxgenerations, self.j)
    self.x.create_output_file()
    self.fitness.append(self.x.getFitness())
    self.x.outputAllFitness(self.result.variables, self.i)

    #print(algorithm.get_TS_result())

  def output(self):    
    call = lastOutput(self.fitness_log, self.result, self.run)
    call.plotMap()
    call.printMap()
                                                                                

#Run Algorithm

In [ ]:
if __name__ == '__main__':
  # Call TSP Execution 
  TSP(
    maxgenerations = 5000,
    popsize = 30, 
    offspring = 30,
    mut_prob = 0, 
    cross_prob = 0.9,
    runs = 30
  )

  '''
    qual par de operadores para cada filho
    contador de pares
  '''